# Examining RB Health in the Proceeding Season Based on Workload

## Get Data

In [1]:
import pandas as pd
import nfl_data_py as nfl

In [2]:
years = range(1999, 2024 + 1)
s_type = "ALL"
df_seasons_all = nfl.import_seasonal_data(years, s_type)

In [3]:
df_roster = nfl.import_seasonal_rosters(years)

In [4]:
df_roster.columns

Index(['season', 'team', 'position', 'depth_chart_position', 'jersey_number',
       'status', 'player_name', 'first_name', 'last_name', 'birth_date',
       'height', 'weight', 'college', 'player_id', 'espn_id', 'sportradar_id',
       'yahoo_id', 'rotowire_id', 'pff_id', 'pfr_id', 'fantasy_data_id',
       'sleeper_id', 'years_exp', 'headshot_url', 'esb_id', 'gsis_it_id',
       'smart_id', 'entry_year', 'rookie_year', 'draft_club', 'ngs_position',
       'week', 'game_type', 'status_description_abbr', 'football_name',
       'draft_number', 'age'],
      dtype='object')

In [5]:
df_roster_clean = df_roster[["season", "player_id", "player_name", "age"]]

In [6]:
df_roster_clean.query("player_id == '00-0034844'")

,season,player_id,player_name,age
3083,2018,00-0034844,Saquon Barkley,21.0
2316,2019,00-0034844,Saquon Barkley,22.0
1939,2020,00-0034844,Saquon Barkley,23.0
1497,2021,00-0034844,Saquon Barkley,24.0
1173,2022,00-0034844,Saquon Barkley,25.0
867,2023,00-0034844,Saquon Barkley,26.0
683,2024,00-0034844,Saquon Barkley,27.0


## Create DFs

In [7]:
df_carries = df_seasons_all.groupby(['player_id', 'season'], as_index=False).agg({
    'carries': 'sum',
    'receptions': 'sum',
    'games': 'max'
})
df_carries["touches"] = df_carries["carries"] + df_carries["receptions"]

In [8]:
df_carries_over_400 = df_carries.query("touches >= 400")
df_carries_over_400.head()

,player_id,season,carries,receptions,games,touches
91,00-0000745,2002,330,74,17,404
94,00-0000745,2005,370,57,17,427
95,00-0000745,2006,353,60,17,413
645,00-0004304,2004,410,24,18,434
769,00-0005092,2001,324,97,17,421


In [9]:
df_rush_all = df_seasons_all.query("season_type == 'REG'")[["player_id", "season", "season_type", "carries", 
                                                            "rushing_yards", "rushing_tds", "rushing_first_downs", 
                                                            "rushing_epa", "games", "fantasy_points_ppr", "fantasy_points",
                                                            "receptions", "receiving_yards", "receiving_tds"
                                                           ]]
df_rush_all["touches"] = df_rush_all["carries"] + df_rush_all["receptions"]

In [10]:
df_merged_one = df_rush_all.merge(
    df_roster_clean[['player_id', 'season', 'player_name', 'age']],
    on=['player_id', 'season'],
    how='left'
)

## Over 400 Carries

In [11]:
columns = [
    "player_name",
    "prev_season",
    "next_season",
    "age_prev",
    "age_next",
    "games_diff",
    "fantasy_points_ppr_diff",
    "fantasy_points_diff",
    "touches_diff",
    "rushing_yards",
    "rushing_tds",
    "rushing_first_downs",
    "receptions",
    "receiving_yards",
    "receiving_tds"
]

df_400 = pd.DataFrame(columns=columns)

In [12]:
data = []
for index, row in df_carries_over_400.iterrows():
    season = row["season"]
    player_id = row["player_id"]

    df_one = df_merged_one.query(f"player_id == '{player_id}' and season == {season}")
    df_two = df_merged_one.query(f"player_id == '{player_id}' and season == {season + 1}")
    
    player = df_one["player_name"].iloc[0]
    age = df_one["age"].iloc[0]

    if len(df_one) == 1 and len(df_two) == 1:
        games_0 = df_one['games'].iloc[0]
        games_1 = df_two['games'].iloc[0]
        games_diff = games_1 - games_0


        yards_0 = df_one['rushing_yards'].iloc[0]
        yards_1 = df_two['rushing_yards'].iloc[0]
        yards_diff = yards_1 - yards_0


        ff_0 = df_one["fantasy_points_ppr"].iloc[0]
        ff_1 = df_two["fantasy_points_ppr"].iloc[0]
        ff_diff = ff_1 - ff_0

        ff_reg_0 = df_one["fantasy_points"].iloc[0]
        ff_reg_1 = df_two["fantasy_points"].iloc[0]
        ff_reg_diff = ff_reg_1 - ff_reg_0

        carries_0 = df_one["touches"].iloc[0]
        carries_1 = df_two["touches"].iloc[0]
        carries_diff = carries_1 - carries_0

        rushing_tds_0 = df_one["rushing_tds"].iloc[0]
        rushing_tds_1 = df_two["rushing_tds"].iloc[0]
        rushing_tds_diff = rushing_tds_1 - rushing_tds_0
        
        rushing_first_downs_0 = df_one["rushing_first_downs"].iloc[0]
        rushing_first_downs_1 = df_two["rushing_first_downs"].iloc[0]
        rushing_first_downs_diff = rushing_first_downs_1 - rushing_first_downs_0
        
        receptions_0 = df_one["receptions"].iloc[0]
        receptions_1 = df_two["receptions"].iloc[0]
        receptions_diff = receptions_1 - receptions_0
        
        receiving_yards_0 = df_one["receiving_yards"].iloc[0]
        receiving_yards_1 = df_two["receiving_yards"].iloc[0]
        receiving_yards_diff = receiving_yards_1 - receiving_yards_0
        
        receiving_tds_0 = df_one["receiving_tds"].iloc[0]
        receiving_tds_1 = df_two["receiving_tds"].iloc[0]
        receiving_tds_diff = receiving_tds_1 - receiving_tds_0

        data_to_add = {
            "player_name": player,
            "prev_season": season,
            "next_season": season + 1,
            "age_prev": age,
            "age_next": age + 1,
            "games_diff": games_diff,
            "fantasy_points_ppr_diff": round(ff_diff, 2),
            "fantasy_points_diff": round(ff_reg_diff, 2),
            "touches_diff": carries_diff,
            "rushing_yards": yards_diff,
            "rushing_tds": rushing_tds_diff,
            "rushing_first_downs": rushing_first_downs_diff,
            "receptions": receptions_diff,
            "receiving_yards": receiving_yards_diff,
            "receiving_tds": receiving_tds_diff
        }
        data.append(data_to_add)

In [13]:
for row in data:
    df_400 = pd.concat([df_400, pd.DataFrame([row])], ignore_index=True)
df_400.head()

/tmp/ipykernel_31980/2357509365.py:2: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_400 = pd.concat([df_400, pd.DataFrame([row])], ignore_index=True)


,player_name,prev_season,next_season,age_prev,age_next,games_diff,fantasy_points_ppr_diff,fantasy_points_diff,touches_diff,rushing_yards,rushing_tds,rushing_first_downs,receptions,receiving_yards,receiving_tds
0,Tiki Barber,2002,2003,27.0,28.0,-1,-76.8,-76.8,-26,-172.0,-9,-1.0,0,-136.0,1
1,Tiki Barber,2005,2006,30.0,31.0,0,-60.3,-64.3,-26,-198.0,-4,6.0,4,-65.0,-2
2,Corey Dillon,2004,2005,29.0,30.0,-5,-71.4,-78.4,-129,-902.0,0,-32.0,7,78.0,0
3,Marshall Faulk,2001,2002,28.0,29.0,-3,-136.7,-133.7,-51,-429.0,-4,-19.0,-3,-228.0,-7
4,Eddie George,1999,2000,25.0,26.0,-3,37.3,35.3,84,178.0,5,10.0,2,-25.0,-2


In [14]:
df_400[["player_name", "prev_season", "age_prev", "fantasy_points_diff", "fantasy_points_ppr_diff", "touches_diff"]]

,player_name,prev_season,age_prev,fantasy_points_diff,fantasy_points_ppr_diff,touches_diff
0,Tiki Barber,2002,27.0,-76.80,-76.80,-26
1,Tiki Barber,2005,30.0,-64.30,-60.30,-26
2,Corey Dillon,2004,29.0,-78.40,-71.40,-129
3,Marshall Faulk,2001,28.0,-133.70,-136.70,-51
4,Eddie George,1999,25.0,35.30,37.30,84
5,Eddie George,2000,26.0,-138.60,-149.60,-94
6,Eddie George,2002,28.0,-76.60,-90.60,-45
7,Ahman Green,2001,24.0,-44.80,-49.80,-23
8,Ahman Green,2003,26.0,-146.40,-156.40,-106
9,Priest Holmes,2003,29.0,-177.10,-232.10,-179


## Over 350 Carries

In [15]:
df_carries_over_350 = df_carries.query("touches >= 350")
df_carries_over_350.head()

,player_id,season,carries,receptions,games,touches
91,00-0000745,2002,330,74,17,404
93,00-0000745,2004,322,52,16,374
94,00-0000745,2005,370,57,17,427
95,00-0000745,2006,353,60,17,413
165,00-0001215,2000,352,13,16,365


In [16]:
columns = [
    "player_name",
    "prev_season",
    "next_season",
    "age_prev",
    "age_next",
    "games_diff",
    "fantasy_points_ppr_diff",
    "fantasy_points_diff",
    "touches_diff",
    "rushing_yards",
    "rushing_tds",
    "rushing_first_downs",
    "receptions",
    "receiving_yards",
    "receiving_tds"
]

df_350 = pd.DataFrame(columns=columns)

In [17]:
data = []
for index, row in df_carries_over_350.iterrows():
    season = row["season"]
    player_id = row["player_id"]

    df_one = df_merged_one.query(f"player_id == '{player_id}' and season == {season}")
    df_two = df_merged_one.query(f"player_id == '{player_id}' and season == {season + 1}")
    
    player = df_one["player_name"].iloc[0]
    age = df_one["age"].iloc[0]

    if len(df_one) == 1 and len(df_two) == 1:
        games_0 = df_one['games'].iloc[0]
        games_1 = df_two['games'].iloc[0]
        games_diff = games_1 - games_0


        yards_0 = df_one['rushing_yards'].iloc[0]
        yards_1 = df_two['rushing_yards'].iloc[0]
        yards_diff = yards_1 - yards_0


        ff_0 = df_one["fantasy_points_ppr"].iloc[0]
        ff_1 = df_two["fantasy_points_ppr"].iloc[0]
        ff_diff = ff_1 - ff_0

        ff_reg_0 = df_one["fantasy_points"].iloc[0]
        ff_reg_1 = df_two["fantasy_points"].iloc[0]
        ff_reg_diff = ff_reg_1 - ff_reg_0

        carries_0 = df_one["touches"].iloc[0]
        carries_1 = df_two["touches"].iloc[0]
        carries_diff = carries_1 - carries_0

        rushing_tds_0 = df_one["rushing_tds"].iloc[0]
        rushing_tds_1 = df_two["rushing_tds"].iloc[0]
        rushing_tds_diff = rushing_tds_1 - rushing_tds_0
        
        rushing_first_downs_0 = df_one["rushing_first_downs"].iloc[0]
        rushing_first_downs_1 = df_two["rushing_first_downs"].iloc[0]
        rushing_first_downs_diff = rushing_first_downs_1 - rushing_first_downs_0
        
        receptions_0 = df_one["receptions"].iloc[0]
        receptions_1 = df_two["receptions"].iloc[0]
        receptions_diff = receptions_1 - receptions_0
        
        receiving_yards_0 = df_one["receiving_yards"].iloc[0]
        receiving_yards_1 = df_two["receiving_yards"].iloc[0]
        receiving_yards_diff = receiving_yards_1 - receiving_yards_0
        
        receiving_tds_0 = df_one["receiving_tds"].iloc[0]
        receiving_tds_1 = df_two["receiving_tds"].iloc[0]
        receiving_tds_diff = receiving_tds_1 - receiving_tds_0

        data_to_add = {
            "player_name": player,
            "prev_season": season,
            "next_season": season + 1,
            "age_prev": age,
            "age_next": age + 1,
            "games_diff": games_diff,
            "fantasy_points_ppr_diff": round(ff_diff, 2),
            "fantasy_points_diff": round(ff_reg_diff, 2),
            "touches_diff": carries_diff,
            "rushing_yards": yards_diff,
            "rushing_tds": rushing_tds_diff,
            "rushing_first_downs": rushing_first_downs_diff,
            "receptions": receptions_diff,
            "receiving_yards": receiving_yards_diff,
            "receiving_tds": receiving_tds_diff
        }
        data.append(data_to_add)

In [18]:
for row in data:
    df_350 = pd.concat([df_350, pd.DataFrame([row])], ignore_index=True)
df_350.head()

/tmp/ipykernel_31980/3075204255.py:2: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_350 = pd.concat([df_350, pd.DataFrame([row])], ignore_index=True)


,player_name,prev_season,next_season,age_prev,age_next,games_diff,fantasy_points_ppr_diff,fantasy_points_diff,touches_diff,rushing_yards,rushing_tds,rushing_first_downs,receptions,receiving_yards,receiving_tds
0,Tiki Barber,2002,2003,27.0,28.0,-1,-76.80,-76.80,-26,-172.0,-9,-1.0,0,-136.0,1
1,Tiki Barber,2004,2005,29.0,30.0,1,11.40,9.40,37,342.0,-4,-5.0,2,-48.0,0
2,Tiki Barber,2005,2006,30.0,31.0,0,-60.30,-64.30,-26,-198.0,-4,6.0,4,-65.0,-2
3,Jerome Bettis,2000,2001,28.0,29.0,-4,-49.62,-44.62,-132,-230.0,-4,-24.0,-5,-49.0,0
4,Stephen Davis,2000,2001,26.0,27.0,1,-33.80,-28.80,27,140.0,-6,4.0,-5,-108.0,0


In [19]:
len(df_350)

134

In [20]:
df_350.query("fantasy_points_diff > 0 or fantasy_points_ppr_diff > 0")[["player_name", "prev_season", "age_prev", 
                                                                        "fantasy_points_diff", "fantasy_points_ppr_diff", "touches_diff"]]

,player_name,prev_season,age_prev,fantasy_points_diff,fantasy_points_ppr_diff,touches_diff
1,Tiki Barber,2004,29.0,9.40,11.40,37
10,Marshall Faulk,1999,26.0,80.20,79.20,20
12,Eddie George,1999,25.0,35.30,37.30,84
14,Eddie George,2001,27.0,86.20,85.20,27
18,Ahman Green,2002,25.0,123.70,116.70,62
20,Priest Holmes,2001,27.0,97.80,105.80,-6
21,Priest Holmes,2002,28.0,0.30,4.30,11
23,Edgerrin James,1999,21.0,27.80,27.80,23
25,Edgerrin James,2003,25.0,40.20,40.20,25
26,Edgerrin James,2004,26.0,11.20,4.20,19


# Over 500

In [21]:
df_carries_over_450 = df_carries.query("touches >= 450")
df_carries_over_450

,player_id,season,carries,receptions,games,touches
927,00-0005883,1999,424,55,20,479
928,00-0005883,2000,425,56,17,481
993,00-0006305,2003,403,58,18,461
1279,00-0008241,2000,404,64,17,468
1597,00-0010442,2004,408,49,18,457
3784,00-0020536,2002,372,79,16,451
4591,00-0021976,2006,429,46,17,475
7764,00-0026796,2012,405,55,18,460
8881,00-0028009,2014,436,61,18,497
13360,00-0034844,2024,436,46,20,482


In [22]:
columns = [
    "player_name",
    "prev_season",
    "next_season",
    "age_prev",
    "age_next",
    "games_diff",
    "fantasy_points_ppr_diff",
    "fantasy_points_diff",
    "touches_diff",
    "rushing_yards",
    "rushing_tds",
    "rushing_first_downs",
    "receptions",
    "receiving_yards",
    "receiving_tds"
]

df_450 = pd.DataFrame(columns=columns)

In [23]:
data = []
for index, row in df_carries_over_450.iterrows():
    season = row["season"]
    player_id = row["player_id"]

    df_one = df_merged_one.query(f"player_id == '{player_id}' and season == {season}")
    df_two = df_merged_one.query(f"player_id == '{player_id}' and season == {season + 1}")
    
    player = df_one["player_name"].iloc[0]
    age = df_one["age"].iloc[0]

    if len(df_one) == 1 and len(df_two) == 1:
        games_0 = df_one['games'].iloc[0]
        games_1 = df_two['games'].iloc[0]
        games_diff = games_1 - games_0


        yards_0 = df_one['rushing_yards'].iloc[0]
        yards_1 = df_two['rushing_yards'].iloc[0]
        yards_diff = yards_1 - yards_0


        ff_0 = df_one["fantasy_points_ppr"].iloc[0]
        ff_1 = df_two["fantasy_points_ppr"].iloc[0]
        ff_diff = ff_1 - ff_0

        ff_reg_0 = df_one["fantasy_points"].iloc[0]
        ff_reg_1 = df_two["fantasy_points"].iloc[0]
        ff_reg_diff = ff_reg_1 - ff_reg_0

        carries_0 = df_one["touches"].iloc[0]
        carries_1 = df_two["touches"].iloc[0]
        carries_diff = carries_1 - carries_0

        rushing_tds_0 = df_one["rushing_tds"].iloc[0]
        rushing_tds_1 = df_two["rushing_tds"].iloc[0]
        rushing_tds_diff = rushing_tds_1 - rushing_tds_0
        
        rushing_first_downs_0 = df_one["rushing_first_downs"].iloc[0]
        rushing_first_downs_1 = df_two["rushing_first_downs"].iloc[0]
        rushing_first_downs_diff = rushing_first_downs_1 - rushing_first_downs_0
        
        receptions_0 = df_one["receptions"].iloc[0]
        receptions_1 = df_two["receptions"].iloc[0]
        receptions_diff = receptions_1 - receptions_0
        
        receiving_yards_0 = df_one["receiving_yards"].iloc[0]
        receiving_yards_1 = df_two["receiving_yards"].iloc[0]
        receiving_yards_diff = receiving_yards_1 - receiving_yards_0
        
        receiving_tds_0 = df_one["receiving_tds"].iloc[0]
        receiving_tds_1 = df_two["receiving_tds"].iloc[0]
        receiving_tds_diff = receiving_tds_1 - receiving_tds_0

        data_to_add = {
            "player_name": player,
            "prev_season": season,
            "next_season": season + 1,
            "age_prev": age,
            "age_next": age + 1,
            "games_diff": games_diff,
            "fantasy_points_ppr_diff": round(ff_diff, 2),
            "fantasy_points_diff": round(ff_reg_diff, 2),
            "touches_diff": carries_diff,
            "rushing_yards": yards_diff,
            "rushing_tds": rushing_tds_diff,
            "rushing_first_downs": rushing_first_downs_diff,
            "receptions": receptions_diff,
            "receiving_yards": receiving_yards_diff,
            "receiving_tds": receiving_tds_diff
        }
        data.append(data_to_add)

In [24]:
for row in data:
    df_450 = pd.concat([df_450, pd.DataFrame([row])], ignore_index=True)
df_450

/tmp/ipykernel_31980/853505256.py:2: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_450 = pd.concat([df_450, pd.DataFrame([row])], ignore_index=True)


,player_name,prev_season,next_season,age_prev,age_next,games_diff,fantasy_points_ppr_diff,fantasy_points_diff,touches_diff,rushing_yards,rushing_tds,rushing_first_downs,receptions,receiving_yards,receiving_tds
0,Eddie George,1999,2000,25.0,26.0,-3,37.30,35.30,84,178.0,5,10.0,2,-25.0,-2
1,Eddie George,2000,2001,26.0,27.0,-1,-149.60,-138.60,-94,-516.0,-9,-38.0,-11,-150.0,-2
2,Ahman Green,2003,2004,26.0,27.0,-2,-156.40,-146.40,-106,-720.0,-8,-40.0,-10,-92.0,-4
3,Edgerrin James,2000,2001,22.0,23.0,-11,-262.60,-225.60,-269,-1028.0,-10,-59.0,-37,-388.0,-5
4,Curtis Martin,2004,2005,31.0,32.0,-6,-181.90,-164.90,-168,-962.0,-7,-55.0,-17,-127.0,-2
5,LaDainian Tomlinson,2002,2003,23.0,24.0,0,59.64,38.64,-38,-38.0,-1,-9.0,21,236.0,3
6,Larry Johnson,2006,2007,26.0,27.0,-9,-242.40,-231.40,-269,-1230.0,-14,-67.0,-11,-224.0,-1
7,Arian Foster,2012,2013,26.0,27.0,-10,-193.60,-175.60,-248,-882.0,-14,-47.0,-18,-34.0,-1
8,DeMarco Murray,2014,2015,26.0,27.0,-3,-166.70,-153.70,-212,-1143.0,-7,-38.0,-13,-94.0,1


# Graphs

In [25]:
import plotly.io as pio

In [26]:
def save_table(df, file_name):
    fig = dict(
        type='table',
        header=dict(values=list(df.columns), align='center', fill_color='lightblue'),
        cells=dict(values=[df[col] for col in df.columns], align='center', fill_color='lightgrey')
    )
    pio.write_html(fig, file=f'{file_name}.html', auto_open=True)

In [27]:
# df_400
# df_400[["player_name", "prev_season", "age_prev", "fantasy_points_diff", "fantasy_points_ppr_diff", "carries_diff"]]
save_table(df_400.sort_values("fantasy_points_diff", ascending=False), "full_400_touches")
save_table(df_400[["player_name", "prev_season", "age_prev", 
                   "fantasy_points_diff", "fantasy_points_ppr_diff", "touches_diff"]].sort_values("fantasy_points_diff", ascending=False), "short_400_touches")

In [28]:
# df_400
# df_400[["player_name", "prev_season", "age_prev", "fantasy_points_diff", "fantasy_points_ppr_diff", "carries_diff"]]
save_table(df_350.sort_values("fantasy_points_diff", ascending=False), "full_350_touches")
save_table(df_350[["player_name", "prev_season", "age_prev", 
                   "fantasy_points_diff", "fantasy_points_ppr_diff", "touches_diff"]].sort_values("fantasy_points_diff", ascending=False), "short_350_touches")

In [29]:
# df_400
# df_400[["player_name", "prev_season", "age_prev", "fantasy_points_diff", "fantasy_points_ppr_diff", "carries_diff"]]
save_table(df_450.sort_values("fantasy_points_diff", ascending=False), "full_450_touches")
save_table(df_450[["player_name", "prev_season", "age_prev", 
                   "fantasy_points_diff", "fantasy_points_ppr_diff", "touches_diff"]].sort_values("fantasy_points_diff", ascending=False), "short_450_touches")

In [30]:
def stats(df):
    total = len(df)
    positive = len(df.query("fantasy_points_diff > 0"))
    print(f"Total: {total}")
    print(f"+: {positive}")
    print(f"% pos: {round(100 * positive/total, 2)}%")
    print()

In [31]:
stats(df_450)
stats(df_400)
stats(df_350)

Total: 9
+: 2
% pos: 22.22%

Total: 49
+: 13
% pos: 26.53%

Total: 134
+: 38
% pos: 28.36%

